In [1]:
%load_ext autotime
import pandas as pd
import os
import geopandas as gpd
from glob import glob
from shapely.validation import make_valid
from tqdm.auto import tqdm
import time

time: 339 ms (started: 2024-08-28 12:04:10 +12:00)


In [3]:
files = pd.DataFrame({"filepath": glob("Asset Directory/**/*.shp", recursive=True)})
files["filename"] = files["filepath"].apply(lambda x: os.path.basename(x).replace(".shp", ""))
files["filesize_GB"] = files["filepath"].apply(lambda x: os.path.getsize(x) / 1e9)
files

,filepath,filename,filesize_GB
0,Asset Directory/LUCAS NZ Land Use/lucas-nz-lan...,lucas-nz-land-use-map-1990-2008-2012-2016-v011,1.435426
1,Asset Directory/lds-new-zealand-17layers-SHP/l...,location-and-extent-of-nzs-aquifers-2015,0.004463
2,Asset Directory/lds-new-zealand-17layers-SHP/n...,nz-railway-centre-lines,0.001587
3,Asset Directory/lds-new-zealand-17layers-SHP/n...,nz-monument-points-topo-150k,0.000008
4,Asset Directory/lds-new-zealand-17layers-SHP/n...,nz-track-centrelines-topo-150k,0.074558
5,Asset Directory/lds-new-zealand-17layers-SHP/l...,linz-managed-crown-property,0.041728
6,Asset Directory/lds-new-zealand-17layers-SHP/n...,nz-drain-centrelines-topo-150k,0.006491
7,Asset Directory/lds-new-zealand-17layers-SHP/n...,nz-rail-station-points-topo-150k,0.000005
8,Asset Directory/lds-new-zealand-17layers-SHP/n...,nz-telephone-centrelines-topo-150k,0.000062
9,Asset Directory/lds-new-zealand-17layers-SHP/n...,nz-facilities,0.000758


time: 12.2 ms (started: 2024-08-28 12:04:36 +12:00)


In [5]:
df = gpd.read_file("Projections/NorthIsland_projection_output_polygon.shp")
df.geometry = df.geometry.apply(make_valid)
poly = df.unary_union

/tmp/ipykernel_29075/795027752.py:3: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  poly = df.unary_union


time: 7.51 s (started: 2024-08-28 12:06:04 +12:00)


In [6]:
for row in tqdm(files.itertuples(index=False), total=len(files)):
  print(f"Now processing {row.filename}")
  start = time.time()
  asset = gpd.read_file(row.filepath)#, engine="pyogrio", use_arrow=True)
  print(f"Time to read file: {time.time() - start}")
  start = time.time()
  clipped_asset = gpd.clip(asset, poly)
  print(f"Time to clip: {time.time() - start}")
  clipped_asset["affected_area"] = clipped_asset.area
  clipped_asset["affected_length"] = clipped_asset.length
  clipped_asset.to_file("affected_assets.gpkg", layer=row.filename, driver='GPKG')

  0%|          | 0/43 [00:00<?, ?it/s]

Now processing lucas-nz-land-use-map-1990-2008-2012-2016-v011
Time to read file: 20.823596239089966
Time to clip: 88.94842219352722
Now processing location-and-extent-of-nzs-aquifers-2015
Time to read file: 1.016951084136963
Time to clip: 1.8867909908294678
Now processing nz-railway-centre-lines
Time to read file: 0.032910823822021484
Time to clip: 0.2139301300048828
Now processing nz-monument-points-topo-150k
Time to read file: 0.0156710147857666
Time to clip: 0.0378108024597168
Now processing nz-track-centrelines-topo-150k


/home/nyou045/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/nyou045/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Time to read file: 0.7532789707183838
Time to clip: 1.9930295944213867
Now processing linz-managed-crown-property
Time to read file: 0.31551599502563477
Time to clip: 1.1461431980133057
Now processing nz-drain-centrelines-topo-150k
Time to read file: 0.20521974563598633
Time to clip: 0.4930107593536377
Now processing nz-rail-station-points-topo-150k
Time to read file: 0.034039974212646484
Time to clip: 0.03805136680603027
Now processing nz-telephone-centrelines-topo-150k
Time to read file: 0.00994873046875
Time to clip: 0.06313538551330566
Now processing nz-facilities
Time to read file: 0.042840003967285156
Time to clip: 0.35218262672424316
Now processing nz-primary-road-parcels
Time to read file: 3.404628276824951
Time to clip: 33.49097156524658
Now processing nz-windmill-points-topo-150k
Time to read file: 0.2528975009918213
Time to clip: 0.08477258682250977
Now processing nz-powerline-centrelines-topo-150k
Time to read file: 0.045198678970336914
Time to clip: 0.09839153289794922
Now

/home/nyou045/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/nyou045/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


...

Now processing DOC_Marine_Reserves
Time to read file: 0.03457975387573242
Time to clip: 0.21100854873657227
Now processing Ecological_Districts
Time to read file: 0.027204513549804688
Time to clip: 1.1812403202056885
Now processing nz-orchard-polygons-topo-150k
Time to read file: 0.03131890296936035
Time to clip: 0.4515557289123535
Now processing nz-cadastral-survey-network-marks
Time to read file: 8.7764151096344
Time to clip: 0.9052865505218506
Now processing nz-trig-points-topo-150k


/home/nyou045/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/nyou045/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Time to read file: 0.4349842071533203
Time to clip: 0.06338953971862793
Now processing protected-areas


/home/nyou045/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/nyou045/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Time to read file: 0.905555248260498
Time to clip: 6.406818151473999
Now processing nz-vineyard-polygons-topo-150k
Time to read file: 0.054325103759765625
Time to clip: 0.2987987995147705
Now processing nz-walking-and-biking-tracks
Time to read file: 0.6189541816711426
Time to clip: 3.356677293777466
Now processing new-zealand-2022-estimated-resident-population-grid-250-metr
Time to read file: 0.9933273792266846
Time to clip: 31.138091325759888
Now processing nz-swamp-polygons-topo-150k
Time to read file: 0.21369028091430664
Time to clip: 0.7891602516174316
Now processing nz-mangrove-polygons-topo-150k
Time to read file: 0.04337787628173828
Time to clip: 0.3609764575958252
Now processing smap-soil-drainage-dec-2023
Time to read file: 18.922531366348267
Time to clip: 28.008750677108765
Now processing nzlri-land-use-capability-2021
Time to read file: 4.953584432601929
Time to clip: 19.24459195137024
time: 11min 45s (started: 2024-08-28 12:06:31 +12:00)


In [7]:
gpd.list_layers("affected_assets.gpkg")

,name,geometry_type
0,lucas-nz-land-use-map-1990-2008-2012-2016-v011,MultiPolygon
1,location-and-extent-of-nzs-aquifers-2015,MultiPolygon
2,nz-railway-centre-lines,MultiLineString
3,nz-monument-points-topo-150k,Point
4,nz-track-centrelines-topo-150k,MultiLineString
5,linz-managed-crown-property,MultiPolygon
6,nz-drain-centrelines-topo-150k,MultiLineString
7,nz-rail-station-points-topo-150k,Unknown
8,nz-telephone-centrelines-topo-150k,Unknown
9,nz-facilities,Polygon


time: 38.6 ms (started: 2024-08-28 12:18:27 +12:00)
